In [ ]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                              options=[
                                                  {'label':'All Sites','value':'ALL'},
                                                  {'label':'CCAFS LC-40','value':'CCAFS LC-40'},
                                                  {'label':'VAFB SLC-4E','value':'VAFB SLC-4E'},
                                                  {'label':'KSC LC-39A','value':'KSC LC-39A'},
                                                  {'label':'CCAFS SLC-40','value':'CCAFS SLC-40'}]
                                              ,value='ALL',
                                              placeholder='ALL',
                                              searchable=True
                                            ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                min=0, max=10000, step=1000,
                                marks={0: '0',
                                2500:'2500',
                                5000:'5000',
                                7500:'7500',
                                10000: '10000'},
                                value=[2500, 7500]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    filtered_df = spacex_df
    ddff=spacex_df.groupby('Launch Site')
    all=ddff['class'].mean()
    a=all.index.values

    name=[]
    values=[]
    valuesneg=[]
    index=0
    for i in a :
        name.append(i)
        values.append(all[i])
        valuesneg.append(1-all[i])
    data = {'Launch site':name,'0': valuesneg, '1': values}
    dat=pd.DataFrame.from_dict(data)
    y=[0,1]
    if entered_site == 'ALL':
        dff = filtered_df.groupby('Launch Site').mean()
        x=['CCAFS LC-40','VAFB SLC-4E','KSC LC-39A','CCAFS SLC-40']
        fig = px.pie(data_frame=dff,values='class',names=x,title='Total success launches by site')
        return fig
    if entered_site == 'CCAFS LC-40':
        fig = px.pie(dat,values=list(dat.loc[0]),title='Total successful launches from CCAFS LC-40')
        return fig
    if entered_site == 'VAFB SLC-4E':     
        fig = px.pie(dat,values=list(dat.loc[1]),title='Total successful launches from VAFB SLC-4E')
        return fig
    if entered_site == 'KSC LC-39A':
        fig = px.pie(dat,values=list(dat.iloc[2]),title='Total successful launches from KSC LC-39A')
        return fig
    if entered_site == 'CCAFS SLC-40':
        fig = px.pie(dat,values=list(dat.iloc[3]),title='Total successful launches from CCAFS SLC-40')
        return fig

        
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'), Input(component_id="payload-slider", component_property="value")] )
def get_scatter(entered_site,slider_range):
    df = spacex_df
    low, high = slider_range
    mask = (df["Payload Mass (kg)"] > low) & (df["Payload Mass (kg)"] < high)
    if entered_site == 'ALL':
        fig = px.scatter(df[mask], x="Payload Mass (kg)", y="class", 
        color='Booster Version Category',title='correlation between payload and success at all sites')
        return fig
    if entered_site == 'CCAFS LC-40':
        mask = (df["Payload Mass (kg)"] > low) & (df["Payload Mass (kg)"] < high) & (df["Launch Site"]=='CCAFS LC-40')
        fig = px.scatter(df[mask], x="Payload Mass (kg)", y="class", 
        color='Booster Version Category',title='correlation between payload and success at CCAFS LC-40')
        return fig
    if entered_site == 'VAFB SLC-4E':
        mask = (df["Payload Mass (kg)"] > low) & (df["Payload Mass (kg)"] < high) & (df["Launch Site"]=='VAFB SLC-4E')
        fig = px.scatter(df[mask], x="Payload Mass (kg)", y="class", 
        color='Booster Version Category',title='correlation between payload and success at VAFB SLC-4E')
        return fig
    if entered_site == 'KSC LC-39A':
        mask = (df["Payload Mass (kg)"] > low) & (df["Payload Mass (kg)"] < high) & (df["Launch Site"]=='KSC LC-39A')
        fig = px.scatter(df[mask], x="Payload Mass (kg)", y="class", 
        color='Booster Version Category',title='correlation between payload and success at KSC LC-39A')
        return fig
    if entered_site == 'CCAFS SLC-40':
        mask = (df["Payload Mass (kg)"] > low) & (df["Payload Mass (kg)"] < high) & (df["Launch Site"]=='CCAFS SLC-40')
        fig = px.scatter(df[mask], x="Payload Mass (kg)", y="class", 
        color='Booster Version Category',title='correlation between payload and success at CCAFS SLC-40')
        return fig


# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output


# Run the app
if __name__ == '__main__':
    app.run_server()


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Mar/2022 07:52:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2022 07:52:47] "GET /apple-touch-icon-precomposed.png HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2022 07:52:47] "GET /_favicon.ico?v=1.19.0 HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2022 07:52:47] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2022 07:52:47] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2022 07:52:47] "GET /apple-touch-icon.png HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2022 07:52:47] "GET /_dash-component-suites/dash_core_components/async-slider.v1_15_0m1611086576.js HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2022 07:52:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2022 07:52:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2022 07:53:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Mar/2022 07:53:06] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0